In [1]:
import os
from keras.preprocessing import image
import numpy as np
from keras.models import Sequential
from keras.layers import Dropout,Conv2D,Flatten,Dense
import cv2
import pygame
from pygame import mixer

Using TensorFlow backend.


pygame 2.3.0 (SDL 2.24.2, Python 3.7.0)
Hello from the pygame community. https://www.pygame.org/contribute.html


In [2]:
def generator(dir, gen=image.ImageDataGenerator(rescale=1./255), shuffle=True,batch_size=1,target_size=(24,24),class_mode='categorical' ):

    return gen.flow_from_directory(dir,batch_size=batch_size,shuffle=shuffle,color_mode='grayscale',class_mode=class_mode,target_size=target_size)

In [3]:
BS= 32
TS=(24,24)
train_batch= generator('dataset_new/train',shuffle=True, batch_size=BS,target_size=TS)
valid_batch= generator('dataset_new/test',shuffle=True, batch_size=BS,target_size=TS)
SPE= len(train_batch.classes)//BS
VS = len(valid_batch.classes)//BS
print(SPE,VS)

img,labels= next(train_batch)
print(img.shape)

Found 2467 images belonging to 4 classes.
Found 433 images belonging to 4 classes.
77 13
(32, 24, 24, 1)


In [5]:
model = Sequential([
    Conv2D(32, kernel_size=(3, 3), activation='relu', input_shape=(24,24,1)),
    Conv2D(32,(3,3),activation='relu'),
    Conv2D(64, (3, 3), activation='relu'),
    Dropout(0.25),
    Flatten(),
    Dense(128, activation='relu'), 
    Dropout(0.5),
    Dense(4, activation='softmax')
])

model.compile(optimizer='adam',loss='categorical_crossentropy',metrics=['accuracy'])

model.fit_generator(train_batch, validation_data=valid_batch,epochs=10,steps_per_epoch=SPE ,validation_steps=VS)


Epoch 1/10
77/77 [==============================] - 24s 313ms/step - loss: 0.6918 - accuracy: 0.6768 - val_loss: 0.4190 - val_accuracy: 0.7596
Epoch 2/10
77/77 [==============================] - 25s 328ms/step - loss: 0.4103 - accuracy: 0.8045 - val_loss: 0.4775 - val_accuracy: 0.8055
Epoch 3/10
77/77 [==============================] - 24s 315ms/step - loss: 0.3499 - accuracy: 0.8283 - val_loss: 0.3636 - val_accuracy: 0.8379
Epoch 4/10
77/77 [==============================] - 22s 281ms/step - loss: 0.3223 - accuracy: 0.8444 - val_loss: 0.2999 - val_accuracy: 0.8105
Epoch 5/10
77/77 [==============================] - 20s 262ms/step - loss: 0.2807 - accuracy: 0.8715 - val_loss: 0.2359 - val_accuracy: 0.8454
Epoch 6/10
77/77 [==============================] - 20s 254ms/step - loss: 0.2479 - accuracy: 0.8871 - val_loss: 0.3262 - val_accuracy: 0.8603
Epoch 7/10
77/77 [==============================] - 20s 262ms/step - loss: 0.2180 - accuracy: 0.9064 - val_loss: 0.2462 - val_accuracy: 0.8653

In [6]:
model.summary

<bound method Network.summary of <keras.engine.sequential.Sequential object at 0x7fa00894deb8>>

In [7]:
mixer.init()
sound = mixer.Sound('alarm.wav')
face = cv2.CascadeClassifier(
    'haar cascade files/haarcascade_frontalface_alt.xml')
leye = cv2.CascadeClassifier(
    'haar cascade files/haarcascade_lefteye_2splits.xml')
reye = cv2.CascadeClassifier(
    'haar cascade files/haarcascade_righteye_2splits.xml')


In [8]:
lbl = ['Close', 'Open']
path = os.getcwd()
cap = cv2.VideoCapture(0)
font = cv2.FONT_HERSHEY_COMPLEX_SMALL
count = 0
score = 0
thicc = 2
rpred = [99]
lpred = [99]

In [9]:
while(True):
    ret, frame = cap.read()
    height, width = frame.shape[:2]
    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    faces = face.detectMultiScale(gray, minNeighbors=5, scaleFactor=1.1, minSize=(25, 25))
    left_eye = leye.detectMultiScale(gray)
    right_eye = reye.detectMultiScale(gray)

    cv2.rectangle(frame, (0, height-50), (200, height),(0, 0, 0), thickness=cv2.FILLED)

    for (x, y, w, h) in faces:
        cv2.rectangle(frame, (x, y), (x+w, y+h), (100, 100, 100), 1)

    for (x, y, w, h) in right_eye:
        r_eye = frame[y:y+h, x:x+w]
        count = count+1
        r_eye = cv2.cvtColor(r_eye, cv2.COLOR_BGR2GRAY)
        r_eye = cv2.resize(r_eye, (24, 24))
        r_eye = r_eye/255
        r_eye = r_eye.reshape(24, 24, -1)
        r_eye = np.expand_dims(r_eye, axis=0)
        rpred = model.predict_classes(r_eye)
        if(rpred[0] == 1):
            lbl = 'Open'
        elif(rpred[0] == 0):
            lbl = 'Closed'
        break

    for (x, y, w, h) in left_eye:
        l_eye = frame[y:y+h, x:x+w]
        count = count+1
        l_eye = cv2.cvtColor(l_eye, cv2.COLOR_BGR2GRAY)
        l_eye = cv2.resize(l_eye, (24, 24))
        l_eye = l_eye/255
        l_eye = l_eye.reshape(24, 24, -1)
        l_eye = np.expand_dims(l_eye, axis=0)
        lpred = model.predict_classes(l_eye)
        if(lpred[0] == 1):
            lbl = 'Open'
        elif(lpred[0] == 0):
            lbl = 'Closed'
        break
    
    if(rpred[0] == 0 and lpred[0] == 0):
        score = score+1
        cv2.putText(frame, "Closed", (10, height-20), font, 1, (255,255,255),1,cv2.LINE_AA)
    # if(rpred[0]==1 or lpred[0]==1):
    else:
        score = score-1
        cv2.putText(frame, "Open", (10, height-20), font, 1, (255,255,255),1,cv2.LINE_AA)

    if(score < 0):
        score = 0
    cv2.putText(frame, 'Score:'+str(score), (100, height-20), font, 1, (255,255,255),1,cv2.LINE_AA)

    if(score > 15):
        # person is feeling sleepy so we beep the alarm
        cv2.imwrite(os.path.join(path, 'image.jpg'), frame)

        try:
            sound.play()
        except:  # isplaying = False
            pass

        if(thicc < 16):
            thicc = thicc+2

        else:
            thicc = thicc-2
            if(thicc < 2):
                thicc = 2

        cv2.rectangle(frame, (0, 0), (width, height),(0,0,255),thicc)
    cv2.imshow('frame', frame)

    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

cap.release()
cv2.destroyAllWindows()